# Example joint control with BLF in Python

## Introduction

### Background

The code in this example is written in Python and uses the [Bipedal Locomotion Framework](https://github.com/ami-iit/bipedal-locomotion-framework) (BLF) to control the joints of the [ergoCub](https://ergocub.eu/project) robot in the Gazebo simulator. To run the code in this example you need to have a working installation of the YARP middleware and the Gazebo simulator, as well as the BLF and ergoCub software. An easy way to install all the dependencies is to use the conda environment that can be created with the following command:

```bash
mamba env create -n blf_example_env gazebo-yarp-plugins icub-models ergocub-software bopedal-locomotion-framework jupyter
```

this environment should contain all the dependencies needed to run the example.

In all terminal windows, activate the conda environment:
```bash
conda activate blf_example_env
```


In [2]:
import numpy as np
import yarp
import tempfile

import bipedal_locomotion_framework.bindings as blf

### Yarp server

Terminal 1:
```bash
export YARP_ROBOT_NAME="ergoCubGazeboV1_1"
yarpserver --write
``` 

This will start the YARP name server that allows the different processes to communicate with each other.

At this point we can connect to the yarp network.

In [4]:
network = yarp.Network()

### Gazebo

Terminal 2:
```bash
export YARP_ROBOT_NAME="ergoCubGazeboV1_1"
export YARP_CLOCK=/clock
gazebo -slibgazebo_yarp_clock.so
```

After Gazebo is open, drag and drop the robot model 'ergoCubGazeboV1_1' from the left panel Insert tab to the Gazebo world.

# Configuration files

A minimal configuration file is shown in the next cell. It has two groups, one for the robot control boards and one for the sensor bridge. The control board group will be used to intialize the access to the joint motors, while the sensor bridge group will be used to access the joint encoders.

In [3]:
config = """
[REMOTE_CONTROL_BOARD]
robot_name ergocubSim

remote_control_boards (left_arm)

joints_list (l_shoulder_pitch l_shoulder_roll l_shoulder_yaw l_elbow l_wrist_yaw l_wrist_pitch l_wrist_roll)

positioning_duration 10.0 # seconds
positioning_tolerance 0.1 # radians
position_direct_max_admissible_error 0.1 # radians

[SENSOR_BRIDGE]
check_for_nan                                    false
stream_joint_states                              true
stream_motor_states                              false
stream_forcetorque_sensors                       false
"""

tmpfile = tempfile.NamedTemporaryFile(mode='w', delete=False)
tmpfile.write(config)
tmpfile.close()

We have now a configuration file in a temporary file, let's use it with the `YarpParametersHandler` class to read the configuration file and print the parameters.

In [4]:
param_handler = blf.parameters_handler.YarpParametersHandler()
param_handler.set_from_filename(tmpfile.name)

# add the local prefix to the remote control board group
# this is needed for the robot control
param_handler.get_group("REMOTE_CONTROL_BOARD").set_parameter_string("local_prefix", "example_controller")

print(param_handler)

 (SENSOR_BRIDGE (check_for_nan false) (stream_joint_states true) (stream_motor_states false) (stream_forcetorque_sensors false)) (REMOTE_CONTROL_BOARD (robot_name ergocubSim) (remote_control_boards (left_arm)) (positioning_duration 10.0) (positioning_tolerance 0.100000000000000005551) (position_direct_max_admissible_error 0.100000000000000005551) (local_prefix example_controller) (joints_list (l_shoulder_pitch l_shoulder_roll l_shoulder_yaw l_elbow l_wrist_yaw l_wrist_pitch l_wrist_roll)))


## Motor Control

To gain access to the robot control board we need to instantiate a PolyDriver object and pass it the configuration file. This will allow us to access the control board remote control interface.

Refs:
 - https://www.yarp.it/latest/classRemoteControlBoardRemapper.html
 - https://ami-iit.github.io/bipedal-locomotion-framework/YarpHelper_8h.html#a2c07f5099140671abde506a8a316e130
 - https://ami-iit.github.io/bipedal-locomotion-framework/structBipedalLocomotion_1_1RobotInterface_1_1PolyDriverDescriptor.html

In [5]:
polydrivers = {}
polydrivers["REMOTE_CONTROL_BOARD"] = blf.robot_interface.construct_remote_control_board_remapper(param_handler.get_group("REMOTE_CONTROL_BOARD"))
polydrivers["REMOTE_CONTROL_BOARD"].is_valid()

[DEBUG] |yarp.dev.PolyDriver|remotecontrolboardremapper| Parameters are (REMOTE_CONTROLBOARD_OPTIONS (writeStrict on)) (axesNames (l_shoulder_pitch l_shoulder_roll l_shoulder_yaw l_elbow l_wrist_yaw l_wrist_pitch l_wrist_roll)) (device remotecontrolboardremapper) (localPortPrefix "/example_controller/remoteControlBoard") (remoteControlBoards ("/ergocubSim/left_arm"))
[DEBUG] |yarp.dev.PolyDriver|remote_controlboard| Parameters are (device remote_controlboard) (local "/example_controller/remoteControlBoard/ergocubSim/left_arm") (remote "/ergocubSim/left_arm") (writeStrict on)
[INFO] |yarp.device.remote_controlboard| RemoteControlBoard is ENABLING the writeStrict option for all commands
[INFO] |yarp.os.Port|/example_controller/remoteControlBoard/ergocubSim/left_arm/rpc:o| Port /example_controller/remoteControlBoard/ergocubSim/left_arm/rpc:o active at tcp://192.168.1.191:10052/
[INFO] |yarp.os.Port|/example_controller/remoteControlBoard/ergocubSim/left_arm/command:o| Port /example_control

True

After instantiating the PolyDriver object we can create a YarpRobotContol object to access the joints specified in the configuration file.

In [6]:
robot_control = blf.robot_interface.YarpRobotControl()
robot_control.initialize(param_handler.get_group("REMOTE_CONTROL_BOARD"))
robot_control.set_driver(polydrivers["REMOTE_CONTROL_BOARD"].poly)

[DEBUG] [2024-06-19 16:22:51.318] [thread: 51905] [blf] [BipedalLocomotion::YarpUtilities::getElementFromSearchable] Missing field named: reading_timeout.
[DEBUG] [2024-06-19 16:22:51.318] [thread: 51905] [blf] [BipedalLocomotion::YarpUtilities::getElementFromSearchable] Missing field named: max_reading_attempts.


True

At this point we have access to the robot joints, for example to count the number of dofs.

In [7]:
n_dof = len(robot_control.get_joint_list())
n_dof

7

We can set the control mode of the joints to position direct and set the desired position of the joints.

In [8]:
robot_control.set_control_mode(blf.robot_interface.YarpRobotControl.PositionDirect)

True

In [9]:
joint_values = np.zeros(n_dof)
control_modes = blf.robot_interface.YarpRobotControl.PositionDirect

robot_control.set_references(joint_values, control_modes)

[ERROR] [2024-06-19 16:22:54.854] [thread: 51905] [blf] [YarpRobotControl::Impl::setReferences] The worst error between the current and the desired position of the joint named 'l_elbow' is greater than 5.729577951308233 deg. Error = 43.96856092656093 deg.


False

If the requested joint position is greater than the parameter `position_direct_max_admissible_error` (in rad) then the `set_reference` function will return false and log an error message.
To avoid this we can set the parameter `position_direct_max_admissible_error` to a higher value, or pass the third argument of the `set_reference` function which is the current joint position read from the encoders.

We can use `check_motion_done` to check if the motion is done. The function returns a tuple of 4 values:
1) isOk: True if the function executed correctly, False otherwise.
2) motionDone: True if the motion ended, False otherwise.
3) isTimeExpired: True if internal timer expired, False otherwise.
4) info: vector containing the list of the joint whose motion did not finish yet, and the corresponding position error.


In [10]:
robot_control.check_motion_done()

(True, True, True, [])

# Sensor Readings



To control the robot in closed loop (hopefully that is the goal) we need to read the joint encoders. We can use the `YarpSensorBridge` class to access the joint encoders.

Let's create a `YarpSensorBridge` object and initialize it with the relative group in the configuration file.

In [11]:
sensor_bridge = blf.robot_interface.YarpSensorBridge()

sensor_bridge.initialize(param_handler.get_group("SENSOR_BRIDGE"))

[INFO] [2024-06-19 16:23:03.600] [thread: 51905] [blf] [YarpSensorBridge::Impl::configureRemoteControlBoardRemapper] The parameter 'joints_list' in not available in the configuration. The order of the joints will be the one passed in RemoteControlBoardRemapper.
[DEBUG] [2024-06-19 16:23:03.600] [thread: 51905] [blf] [BipedalLocomotion::YarpUtilities::getElementFromSearchable] Missing field named: stream_pids.
[DEBUG] [2024-06-19 16:23:03.600] [thread: 51905] [blf] [BipedalLocomotion::YarpUtilities::getElementFromSearchable] Missing field named: stream_motor_PWM.
[DEBUG] [2024-06-19 16:23:03.600] [thread: 51905] [blf] [BipedalLocomotion::YarpUtilities::getElementFromSearchable] Missing field named: stream_inertials.
[DEBUG] [2024-06-19 16:23:03.600] [thread: 51905] [blf] [BipedalLocomotion::YarpUtilities::getElementFromSearchable] Missing field named: stream_cartesian_wrenches.
[DEBUG] [2024-06-19 16:23:03.600] [thread: 51905] [blf] [BipedalLocomotion::YarpUtilities::getElementFromSearc

True

To have access to the joint encoders we need to link the `PolyDriver` object to the `YarpSensorBridge` object.
This will allow us to read encoders of the joints we are controlling and that we defined in the `REMOTE_CONTROL_BOARD` group of the configuration file.

In [12]:
sensor_bridge.set_drivers_list([polydrivers["REMOTE_CONTROL_BOARD"]])

[INFO] [2024-06-19 16:23:05.123] [thread: 51905] [blf] [YarpSensorBridge::Impl::compareControlBoardJointsList] Found all joints with the remapped index
[INFO] [2024-06-19 16:23:05.123] [thread: 51905] [blf] [YarpSensorBridge::Impl::compareControlBoardJointsList] Remapped Index: 0, Joint name: l_shoulder_pitch.
[INFO] [2024-06-19 16:23:05.123] [thread: 51905] [blf] [YarpSensorBridge::Impl::compareControlBoardJointsList] Remapped Index: 1, Joint name: l_shoulder_roll.
[INFO] [2024-06-19 16:23:05.123] [thread: 51905] [blf] [YarpSensorBridge::Impl::compareControlBoardJointsList] Remapped Index: 2, Joint name: l_shoulder_yaw.
[INFO] [2024-06-19 16:23:05.123] [thread: 51905] [blf] [YarpSensorBridge::Impl::compareControlBoardJointsList] Remapped Index: 3, Joint name: l_elbow.
[INFO] [2024-06-19 16:23:05.123] [thread: 51905] [blf] [YarpSensorBridge::Impl::compareControlBoardJointsList] Remapped Index: 4, Joint name: l_wrist_yaw.
[INFO] [2024-06-19 16:23:05.123] [thread: 51905] [blf] [YarpSenso

True

e: l_wrist_pitch.
[INFO] [2024-06-19 16:23:05.123] [thread: 51905] [blf] [YarpSensorBridge::Impl::compareControlBoardJointsList] Remapped Index: 6, Joint name: l_wrist_roll.


At this point we can read the joint encoders and print them.

In [13]:
sensor_bridge.advance()
are_joints_ok, joint_positions, _ = sensor_bridge.get_joint_positions()
joint_positions

array([-5.07991605e-01,  5.07944520e-01,  2.33920528e-03,  7.76957814e-01,
        5.94554546e-05, -3.44187053e-03,  6.03756655e-04])

# Example Control

We now have all the tools to control the robot. We can create a simple control loop that moves the joints to a desired position.

In [14]:
joint_to_test = "l_shoulder_pitch"
joint_index = robot_control.get_joint_list().index(joint_to_test)

In [19]:
for i in range(100):
    t = i * 0.1
    # read the joint positions
    sensor_bridge.advance()
    are_joints_ok, joint_positions, _ = sensor_bridge.get_joint_positions()

    new_joint_positions = joint_positions.copy()
    # set the references
    new_joint_positions[joint_index] += 0.1 * np.cos(10 * t)
    robot_control.set_references(new_joint_positions, blf.robot_interface.YarpRobotControl.PositionDirect, joint_positions)
    yarp.delay(0.1)